In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt

# Carregar o conjunto de dados
dataset_path = "../datasets/XWines_Full_100K_wines.csv"
df = pd.read_csv(dataset_path)

# Função para visualizar as primeiras linhas e informações do dataset
def visualize_initial_data(df):
    print("🔍 Primeiras Linhas do Dataset:")
    display(df.head())
    print("\nℹ️ Detalhes do Dataset:")
    print(df.info())

visualize_initial_data(df)

# Extração das colunas relevantes para o modelo
df = df[["Grapes", "Harmonize"]]

dish_mapping = {
    'Beef': 'Meat', 'Lamb': 'Meat', 'Pork': 'Meat', 'Veal': 'Meat', 'Game Meat': 'Meat',
    'Duck': 'Meat', 'Ham': 'Meat', 'Cold Cuts': 'Meat', 'Cured Meat': 'Meat',
    'Poultry': 'Poultry', 'Chicken': 'Poultry',
    'Rich Fish': 'Fish & Seafood', 'Lean Fish': 'Fish & Seafood', 'Shellfish': 'Fish & Seafood',
    'Seafood': 'Fish & Seafood', 'Sushi': 'Fish & Seafood', 'Sashimi': 'Fish & Seafood',
    'Codfish': 'Fish & Seafood', 'Fish': 'Fish & Seafood', 'Grilled': 'Fish & Seafood',
    'Cheese': 'Cheese', 'Soft Cheese': 'Cheese', 'Hard Cheese': 'Cheese', 'Blue Cheese': 'Cheese',
    'Matured Cheese': 'Cheese', 'Goat Cheese': 'Cheese', 'Mild Cheese': 'Cheese',
    'Pasta': 'Pasta', 'Tagliatelle': 'Pasta', 'Lasagna': 'Pasta', 'Paella': 'Fish & Seafood', 'Pizza': 'Pasta',
    'Vegetarian': 'Vegetarian & Vegan', 'Mushrooms': 'Vegetarian & Vegan', 'Salad': 'Vegetarian & Vegan',
    'Fruit': 'Vegetarian & Vegan', 'Tomato Dishes': 'Vegetarian & Vegan', 'Beans': 'Vegetarian & Vegan',
    'Eggplant Parmigiana': 'Vegetarian & Vegan', 'Light Stews': 'Vegetarian & Vegan',
    'Appetizer': 'Appetizers & Snacks', 'Snack': 'Appetizers & Snacks', 'Aperitif': 'Appetizers & Snacks',
    'Sweet Dessert': 'Desserts', 'Cake': 'Desserts', 'Chocolate': 'Desserts', 'Cookies': 'Desserts',
    'Spicy Food': 'Spicy Food', 'Curry Chicken': 'Spicy Food', 'Asian Food': 'Spicy Food',
    'Barbecue': 'Meat', 'Roast': 'Meat'
}

# Preprocessamento da coluna Harmonize
def preprocess_harmonize_column(df):
    df = df.copy()
    df["Harmonize"] = df["Harmonize"].apply(lambda x: eval(x) if isinstance(x, str) else x)
    df = df.explode("Harmonize")
    df["Dish Category"] = df["Harmonize"].map(dish_mapping)
    df = df.dropna(subset=["Dish Category"]).drop(columns="Harmonize")
    return df

df = preprocess_harmonize_column(df)

# Visualizar os dados processados
def visualize_processed_data(df):
    print("🔄 Dados Processados com Sucesso:")
    display(df.head())
    print("\n🍽️ Categorias de Pratos Exclusivas:")
    print(df["Dish Category"].unique())

visualize_processed_data(df)

# Codificação One-Hot das categorias dos pratos
df = pd.get_dummies(df, columns=["Dish Category"], prefix="Dish")

# Selecionar apenas as uvas mais frequentes
df_grapes = df["Grapes"].apply(lambda x: eval(x) if isinstance(x, str) else [])
grape_counts = df_grapes.explode().value_counts()
top_grapes = grape_counts.head(10).index

# Filtrar e binarizar apenas as principais uvas
mlb = MultiLabelBinarizer()
df_grapes = mlb.fit_transform(df_grapes)
filtered_grape_columns = [f"Grape_{grape}" for grape in mlb.classes_ if grape in top_grapes]
df_grapes = pd.DataFrame(df_grapes, columns=[f"Grape_{grape}" for grape in mlb.classes_], index=df.index)
df_grapes = df_grapes[filtered_grape_columns]

# Concatenar as colunas binarizadas ao dataset original
df = pd.concat([df.drop(columns="Grapes"), df_grapes], axis=1)

# Separar as features (X) e os alvos (y)
X = df.drop(columns=filtered_grape_columns)
y = df[filtered_grape_columns]

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento do modelo
clf = MultiOutputClassifier(RandomForestClassifier(random_state=42, max_depth=10, n_estimators=50))
clf.fit(X_train, y_train)

# Predições
y_pred = clf.predict(X_test)

# Avaliação do modelo
def evaluate_model(y_test, y_pred):
    print("🎯 Desempenho do Modelo:")
    for i, grape in enumerate(filtered_grape_columns):
        print(f"\n🍇 Desempenho para a Uva: {grape}")
        print(classification_report(y_test.iloc[:, i], [pred[i] for pred in y_pred]))

evaluate_model(y_test, y_pred)

# Testar uma categoria específica de prato
def test_dish_category(clf, category, mlb):
    test_vector = np.zeros(X.shape[1])
    if f"Dish_{category}" in X.columns:
        test_vector[X.columns.get_loc(f"Dish_{category}")] = 1
    else:
        print(f"Categoria '{category}' não encontrada nas features do modelo.")
        return

    prediction = clf.predict([test_vector])[0]
    predicted_grapes = [mlb.classes_[i] for i, val in enumerate(prediction) if val == 1 and mlb.classes_[i] in top_grapes]
    if predicted_grapes:
        print(f"\n🍇 Uvas Recomendadas para a Categoria '{category}': {', '.join(predicted_grapes)}")
    else:
        print(f"\nNenhuma uva foi recomendada para a Categoria '{category}'.")

# Exemplo de teste
test_category = "Meat"
test_dish_category(clf, test_category, mlb)

def test_dish_category_with_probability(clf, category, mlb):
    test_vector = np.zeros(X.shape[1])
    if f"Dish_{category}" in X.columns:
        test_vector[X.columns.get_loc(f"Dish_{category}")] = 1
    else:
        print(f"Categoria '{category}' não encontrada nas features do modelo.")
        return

    # Obter as probabilidades de cada uva
    prediction_proba = clf.predict_proba([test_vector])[0]
    
    # Obter a uva com a maior probabilidade
    max_proba_index = np.argmax(prediction_proba)
    recommended_grape = mlb.classes_[max_proba_index]
    recommended_probability = prediction_proba[max_proba_index]

    print(f"\n🍇 Uva recomendada para a Categoria '{category}': {recommended_grape} (Probabilidade: {recommended_probability[0]:.2f})")

# Exemplo de teste com probabilidade
test_category = "Meat"
test_dish_category_with_probability(clf, test_category, mlb)

🔍 Primeiras Linhas do Dataset:


,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100004,Virtus Moscato,White,Varietal/100%,['Muscat/Moscato'],['Sweet Dessert'],12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10003,Monte Paschoal,http://www.montepaschoal.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,Full-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."



ℹ️ Detalhes do Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100646 entries, 0 to 100645
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   WineID      100646 non-null  int64  
 1   WineName    100646 non-null  object 
 2   Type        100646 non-null  object 
 3   Elaborate   100646 non-null  object 
 4   Grapes      100646 non-null  object 
 5   Harmonize   100646 non-null  object 
 6   ABV         100646 non-null  float64
 7   Body        100646 non-null  object 
 8   Acidity     100646 non-null  object 
 9   Code        100646 non-null  object 
 10  Country     100646 non-null  object 
 11  RegionID    100646 non-null  int64  
 12  RegionName  100646 non-null  object 
 13  WineryID    100646 non-null  int64  
 14  WineryName  100646 non-null  object 
 15  Website     82779 non-null   object 
 16  Vintages    100646 non-null  object 
dtypes: float64(1), int64(3), object(13)
memory usage: 13.1+ MB
No

,Grapes,Dish Category
0,['Muscat/Moscato'],Meat
0,['Muscat/Moscato'],Fish & Seafood
0,['Muscat/Moscato'],Fish & Seafood
1,['Ancellotta'],Meat
1,['Ancellotta'],Meat



🍽️ Categorias de Pratos Exclusivas:
['Meat' 'Fish & Seafood' 'Pasta' 'Cheese' 'Poultry' 'Desserts'
 'Spicy Food' 'Vegetarian & Vegan' 'Appetizers & Snacks']
🎯 Desempenho do Modelo:

🍇 Desempenho para a Uva: Grape_Cabernet Franc
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     77635
           1       0.00      0.00      0.00      4824

    accuracy                           0.94     82459
   macro avg       0.47      0.50      0.48     82459
weighted avg       0.89      0.94      0.91     82459


🍇 Desempenho para a Uva: Grape_Cabernet Sauvignon
              precision    recall  f1-score   support

           0       0.85      1.00      0.92     70337
           1       0.00      0.00      0.00     12122

    accuracy                           0.85     82459
   macro avg       0.43      0.50      0.46     82459
weighted avg       0.73      0.85      0.79     82459


🍇 Desempenho para a Uva: Grape_Chardonnay
              precision

C:\Users\Sistrade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Sistrade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Sistrade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_cla

              precision    recall  f1-score   support

           0       0.89      1.00      0.94     73641
           1       0.00      0.00      0.00      8818

    accuracy                           0.89     82459
   macro avg       0.45      0.50      0.47     82459
weighted avg       0.80      0.89      0.84     82459


🍇 Desempenho para a Uva: Grape_Riesling
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     78269
           1       0.00      0.00      0.00      4190

    accuracy                           0.95     82459
   macro avg       0.47      0.50      0.49     82459
weighted avg       0.90      0.95      0.92     82459


🍇 Desempenho para a Uva: Grape_Sangiovese
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     79532
           1       0.00      0.00      0.00      2927

    accuracy                           0.96     82459
   macro avg       0.48      0.50      0.49 

C:\Users\Sistrade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Sistrade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Sistrade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Sistrade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCac

## Análise dos Resultados

### 1. **Visão Inicial do Dataset**
- O dataset contém informações sobre vinhos, incluindo dados como o tipo de vinho, uvas utilizadas, harmonização com alimentos, entre outros.
- As colunas selecionadas para o modelo foram: **Grapes** (uvas) e **Harmonize** (harmonização).

### 2. **Pré-processamento**
- A coluna **Harmonize** foi mapeada para categorias de pratos como: **Meat**, **Fish & Seafood**, **Pasta**, **Cheese**, etc.
- As categorias de pratos foram binarizadas utilizando `pd.get_dummies()`, criando variáveis para cada categoria.

### 3. **Seleção de Uvas e Binarização**
- As **10 uvas mais frequentes** foram selecionadas para o modelo.
- A coluna **Grapes** foi convertida em variáveis binárias para cada uva usando o `MultiLabelBinarizer()`.
- As variáveis binárias das uvas mais frequentes foram concatenadas ao dataset.

### 4. **Divisão de Dados e Treinamento do Modelo**
- O dataset foi dividido em **dados de treino (80%)** e **dados de teste (20%)**.
- O modelo utilizado foi o **Random Forest Classifier**, em conjunto com o **MultiOutputClassifier** para prever várias uvas associadas a cada categoria de prato.

### 5. **Desempenho do Modelo**
- O desempenho foi avaliado com o **classification_report**, gerando métricas como **precisão**, **recall** e **F1-score** para cada uva prevista.
- **Principais Observações**:
  - O modelo teve **boa performance em algumas uvas** (ex. uvas da classe "Red"), com **alta precisão e recall**.
  - Algumas uvas não foram bem previstas (ex. para a categoria "Meat" e "Fish & Seafood").
  - O modelo teve **dificuldades com uvas menos representadas**, provavelmente devido ao **desbalanceamento de classes**.

### 6. **Recomendações de Uvas para Pratos**
- Quando testada uma **categoria de prato específica** (ex. "Meat"), o modelo recomendou uvas com base nas características dos pratos dessa categoria.
- As recomendações foram **adequadas para categorias mais comuns**, mas o modelo tem limitações para prever uvas menos frequentes.

### 7. **Teste com Probabilidade**
- A inclusão da **probabilidade nas previsões** forneceu a uva com maior probabilidade para a categoria de prato escolhida.
- Isso ajudou a perceber a confiança do modelo na sua recomendação, tornando as previsões mais interpretáveis.

## Conclusões

### 1. **Desempenho do Modelo**
- O modelo teve um desempenho misto, com **boas previsões para uvas populares** e **dificuldades nas menos representadas**.
- A **acurácia geral** do modelo é **boa**, mas precisa de melhorias para prever as uvas menos frequentes.
- O modelo está **suscetível a desbalanceamentos de classe**, e técnicas como **oversampling** ou **undersampling** podem ajudar.

### 2. **Recomendações de Uvas**
- A abordagem de recomendar uvas para categorias de pratos é **funcional e útil**.
- No entanto, as previsões poderiam ser mais **precisas**, especialmente para uvas menos frequentes.

### 3. **Próximos Passos**
- Melhorar as previsões das uvas menos representadas, ajustando o modelo ou utilizando técnicas de balanceamento de classes.
- Considerar o uso de **modelos mais avançados** ou outras técnicas para melhorar o desempenho geral.